In [15]:
# Extraccion de texto de página web
# ==============================================================================
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

# Librerías de apoyo
# ==============================================================================
import time
import re
from tqdm import tqdm

# Tratamiento de datos
# ==============================================================================
import pandas as pd

# Visualización Jupyter
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time
chrome_driver_path = "/usr/lib/chromium-browser/chromedriver"
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

CPU times: user 6.78 ms, sys: 5.21 ms, total: 12 ms
Wall time: 2.22 s


In [3]:
%%time
driver.get("https://diariooficial.elperuano.pe/Normas")
time.sleep(3)

CPU times: user 2.61 ms, sys: 0 ns, total: 2.61 ms
Wall time: 4.52 s


In [4]:
%%time
seach_date_begin = driver.find_element_by_id("cddesde")
seach_date_end = driver.find_element_by_id("cdhasta")
seach_date_begin.clear()
seach_date_end.clear()
seach_date_begin.send_keys("20/07/2021")
seach_date_end.send_keys("23/07/2021")
seach_date_end.send_keys(Keys.RETURN)
seach_date_end.send_keys(Keys.RETURN)
time.sleep(3)

CPU times: user 10.8 ms, sys: 157 µs, total: 10.9 ms
Wall time: 3.28 s


In [5]:
%%time
page_source = driver.page_source
pretty_page = BeautifulSoup(page_source, "lxml")
laws = pretty_page.find_all("article", class_="edicionesoficiales_articulos")
len(laws)

CPU times: user 229 ms, sys: 5.22 ms, total: 235 ms
Wall time: 281 ms


295

In [6]:
#url = "https://busquedas.elperuano.pe/normaslegales/ley-que-declara-de-interes-nacional-la-educacion-basica-tec-ley-no-31306-1975873-1/"
#content_x = requests.get(url, verify=False)
#BeautifulSoup(content_x.content).find_all(class_="cuerpo")[7].text

'Artículo 2. Mejora continua'

In [16]:
%%time
data = []

for law in tqdm(laws):
    row = {}
    row["category"] = law.select_one(".ediciones_texto h4").text
    row["title"] = law.select_one(".ediciones_texto h5").text
    row["date"] = law.select_one(".ediciones_texto p b").text.replace("Fecha: ", "").strip()
    row["abstract"] = law.select_one(".ediciones_texto p:nth-of-type(2)").text
    content_url = law.select_one(".ediciones_texto h5 a")["href"]
    content_page = requests.get(content_url, verify=False)
    paragraphs = BeautifulSoup(content_page.content).find_all(class_="cuerpo")
    row["content"] = " ".join([paragraph.text for paragraph in paragraphs])
    #row["content_url"] = law.select_one(".ediciones_texto h5 a")["href"]
    #row["pdf_section"] = law.select_one(".ediciones_botones li:nth-child(1) a")["href"]
    #row["pdf_whole"] = law.select_one(".ediciones_botones li:nth-child(2) a")["href"]
    data.append(row)

100%|██████████| 295/295 [01:55<00:00,  2.54it/s]

CPU times: user 7.3 s, sys: 123 ms, total: 7.43 s
Wall time: 1min 55s


In [17]:
dataset_base = pd.DataFrame(data)

In [18]:
dataset_base.head()

,category,title,date,abstract,content
0,DECRETOS DE URGENCIA,DECRETO DE URGENCIA N° 075-2011,23/07/2021,Decreto de Urgencia que establece medidas extr...,EL PRESIDENTE DE LA REPÚBLICA CONSIDERANDO: Qu...
1,DECRETOS DE URGENCIA,DECRETO DE URGENCIA N° 076-2011,23/07/2021,Decreto de Urgencia que dicta medidas extraord...,EL PRESIDENTE DE LA REPÚBLICA CONSIDERANDO: Qu...
2,SEGURO INTEGRAL DE SALUD,RESOLUCION JEFATURAL N° 095-2021-SIS,23/07/2021,Aprueban Transferencia Financiera del SIS a fa...,"Lima, 22 de julio de 2021 VISTOS: El Informe C..."
3,CORTES SUPERIORES DE JUSTICIA,RESOLUCION ADMINISTRATIVA N° 000746-2021-P-CS...,23/07/2021,Disponen el inicio de las actividades del proy...,"Villa María Del Triunfo, 21 de julio de 2021 V..."
4,CONGRESO DE LA REPUBLICA,LEY N° 31307,23/07/2021,Nuevo Código Procesal Constitucional,LA PRESIDENTA A. I. DEL CONGRESO DE LA REPÚBLI...


In [20]:
dataset_base.content[0]

'EL PRESIDENTE DE LA REPÚBLICA CONSIDERANDO: Que, la Constitución Política del Perú establece en su artículo 7 que todos tienen derecho a la protección de su salud; asimismo, en su artículo 9 dispone que el Estado determina la política nacional de salud, el Poder Ejecutivo norma y supervisa su aplicación, y es responsable de diseñarla y conducirla en forma plural y descentralizadora para facilitar a todos el acceso equitativo a los servicios de salud. Adicionalmente, el artículo 44 de la norma constitucional prevé que son deberes primordiales del Estado garantizar la plena vigencia de los derechos humanos, proteger a la población de las amenazas contra su seguridad, y promover el bienestar general que se fundamenta en la justicia y en el desarrollo integral y equilibrado de la Nación; Que, en la misma línea, los numerales I y II del Título Preliminar de la Ley Nº 26842, Ley General de Salud, establecen que la salud es condición indispensable del desarrollo humano y medio fundamental pa